<a href="https://colab.research.google.com/github/pariyaab/Book_recommendation/blob/master/Book_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a recommendation system with TensorFlow and Keras


Here, we are going to build a practical book recommender service using [TensorFlow Recommenders](https://www.tensorflow.org/recommenders) and [Keras](https://keras.io/) and deploy it using [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving).


In [ ]:
!pip install -q tensorflow==2.0.0

In [3]:
!pip install --upgrade -q tensorflow

     |████████████████████████████████| 578.0 MB 14 kB/s 
     |████████████████████████████████| 1.7 MB 52.4 MB/s 
     |████████████████████████████████| 5.9 MB 38.5 MB/s 
     |████████████████████████████████| 438 kB 47.4 MB/s 


In [4]:
!pip install -q --upgrade tensorflow-datasets
!pip install -q tensorflow-recommenders
!pip install -q scann

     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 89 kB 3.4 MB/s 
     |████████████████████████████████| 10.4 MB 4.5 MB/s 


In [5]:
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [6]:
# Download the data, save them as `tfrecord` files, load the `tfrecord` files
# and create the `tf.data.Dataset` object containing the dataset.
ratings_dataset, ratings_dataset_info = tfds.load(
    name='amazon_us_reviews/Books_v1_02',
    # Books_v1_02 dataset is not splitted into `train` and `test` sets by default.
    # So TFDS has put it all into `train` split. We load it completely and split
    # it manually.
    split='train',
    # `with_info=True` makes the `load` function return a `tfds.core.DatasetInfo`
    # object containing dataset metadata like product_parent, review_body, review_date,
    # review_id, etc.
    with_info=True
)

# Calling the `tfds.load()` function in old versions of TFDS won't return an
# instance of `tf.data.Dataset` type. So we can make sure about it.

assert isinstance(ratings_dataset, tf.data.Dataset)

print(
    "ratings_dataset size: %d" % ratings_dataset.__len__()
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3105520 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/amazon_us_reviews/Books_v1_02/0.1.0.incompleteAFJYVV/amazon_us_reviews-tra…

Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Books_v1_02/0.1.0. Subsequent calls will reuse this data.
ratings_dataset size: 3105520


In [7]:
list(ratings_dataset.take(1).as_numpy_iterator())[0]

{'data': {'customer_id': b'51389465',
  'helpful_votes': 1,
  'marketplace': b'US',
  'product_category': b'Books',
  'product_id': b'1583141642',
  'product_parent': b'591848411',
  'product_title': b'Admission Of Love (Arabesque)',
  'review_body': b"What a great story this was. I'll admit I was skeptical before reading  this book because she is a new author. Thank you for proving me wrong. I  could not put it down! And the love scenes... I had to fan myself!!! If you  don't have this book,drop what you're doing and run out and buy it! This  story was funny and hearwarming!!",
  'review_date': b'2000-08-10',
  'review_headline': b'Oh So HOT!!!!!!!!!!!!',
  'review_id': b'R3MDBY5UZQXZST',
  'star_rating': 5,
  'total_votes': 1,
  'verified_purchase': 1,
  'vine': 1}}

In [8]:
# Use `tfds.as_dataframe()` to convert `tf.data.Dataset` to `pandas.DataFrame`.
# Add the `tfds.core.DatasetInfo` as second argument of `tfds.as_dataframe` to
# load the full dataset in-memory, and can be very expensive to display. So use it only
# with take() function.
print(
    tfds.as_dataframe(ratings_dataset.take(1), ratings_dataset_info)
)


  data/customer_id  data/helpful_votes data/marketplace data/product_category  \
0      b'51389465'                   1            b'US'              b'Books'   

  data/product_id data/product_parent                data/product_title  \
0   b'1583141642'        b'591848411'  b'Admission Of Love (Arabesque)'   

                                    data/review_body data/review_date  \
0  b"What a great story this was. I'll admit I wa...    b'2000-08-10'   

       data/review_headline     data/review_id  data/star_rating  \
0  b'Oh So HOT!!!!!!!!!!!!'  b'R3MDBY5UZQXZST'                 5   

   data/total_votes  data/verified_purchase  data/vine  
0                 1                       1          1  


In [ ]:
## Feature selection
ratings_dataset = ratings_dataset.map(
    lambda rating: {
        # `customer_id` is useful as a user identifier.
        'customer_id': rating['data']['customer_id'],
           # `product_id` is useful as a book identifier.
        'product_id': rating['data']['product_id'],
        # `product_title` is useful as a textual information about the book.
        'product_title': rating['data']['product_title'],
        # `helpful_votes` shows the user's level of interest to a book.
        'helpful_votes': rating['data']['helpful_votes'],
    }
)

In [10]:
list(ratings_dataset.take(2).as_numpy_iterator())

[{'customer_id': b'51389465',
  'product_id': b'1583141642',
  'product_title': b'Admission Of Love (Arabesque)',
  'helpful_votes': 1},
 {'customer_id': b'23641112',
  'product_id': b'0671620991',
  'product_title': b"Solve Your Child's Sleep Problems",
  'helpful_votes': 3}]

In [11]:
## Split dataset randomly (80% for train and 20% for test)
trainset_size = 0.8 * ratings_dataset.__len__().numpy()
# In an industrial recommender system, this would most likely be done by time:
# The data up to time T would be used to predict interactions after T.

# set the global seed:
tf.random.set_seed(42)
# Shuffle the elements of the dataset randomly.
ratings_dataset_shuffled = ratings_dataset.shuffle(
    # the new dataset will be sampled from a buffer window of first `buffer_size`
    # elements of the dataset
    buffer_size=100_000,
    # set the random seed that will be used to create the distribution.
    seed=42,
    # `list(dataset.as_numpy_iterator()` yields different result for each call
    # Because reshuffle_each_iteration defaults to True.
    reshuffle_each_iteration=False
)
ratings_trainset = ratings_dataset_shuffled.take(trainset_size)
ratings_testset = ratings_dataset_shuffled.skip(trainset_size)

print(
    "ratings_trainset size: %d" % ratings_trainset.__len__()
)
print(
    "ratings_testset size: %d" % ratings_testset.__len__()
)

ratings_trainset size: 2484416
ratings_testset size: 621104


## Preprocess raw features and make embeddings with Keras preprocessing layers

Raw features are usually not be immediately usable in a machine learning model and should be preprocessed in the first place.
- **Numerical features** (ratings, prices, timestamps, etc) can be far away in terms of scale and need to be `normalized` so that their values lie in a small interval around 0.
- **Categorical features** (ids, usernames/emails, titles, etc) are usually string features and have to be translated into `embedding vectors` (numerical feature representations) that are adjusted during training the model.
- **Text features** (descriptions, comments, etc) need to be at first, `tokenized` (split into smaller parts such as individual words known as word pieces) and then translated into embeddings.

[Keras preprocessing layers](https://keras.io/guides/preprocessing_layers/) let us build `end-to-end` portable models that accept raw features (raw images or raw structured data) as input; models that handle feature normalization or feature value indexing on their own.

In [12]:
for item in ratings_trainset.take(5).as_numpy_iterator():
  print(item)

{'customer_id': b'52997076', 'product_id': b'0373870574', 'product_title': b'A Family for Andi (Love Inspired #57)', 'helpful_votes': 2}
{'customer_id': b'50796401', 'product_id': b'0375503641', 'product_title': b'Shutterbabe:  Adventures in Love and War', 'helpful_votes': 5}
{'customer_id': b'49586917', 'product_id': b'0316666343', 'product_title': b'The Lovely Bones', 'helpful_votes': 5}
{'customer_id': b'33219914', 'product_id': b'0972995404', 'product_title': b'American Bachelor', 'helpful_votes': 1}
{'customer_id': b'33678582', 'product_id': b'0140177396', 'product_title': b'Of Mice and Men', 'helpful_votes': 0}


### Normalize numerical features
`helpful_votes` values are for rating use to books and it is numerical features and it can be normalized in a small interval around 0. Standardization (`Z-score Normalization`) is a common preprocessing transformation that rescales features to normalize their range by subtracting the feature's `mean` and dividing by its `standard deviation`.


In [ ]:
helpful_votes_layer = tf.keras.layers.experimental.preprocessing.Normalization(axis=None)
helpful_votes_layer.adapt(ratings_trainset.map( 
    lambda rating: rating['helpful_votes']
    )
)

In [14]:
for rating in ratings_trainset.take(3).as_numpy_iterator():
  print(
      f"Raw helpful_votes: {rating['helpful_votes']} ->",
      f"Normalized timestamp: {helpful_votes_layer(rating['helpful_votes'])}"
  )

Raw helpful_votes: 2 -> Normalized timestamp: -0.2795896530151367
Raw helpful_votes: 5 -> Normalized timestamp: -0.16532482206821442
Raw helpful_votes: 5 -> Normalized timestamp: -0.16532482206821442


### Turning categorical features into embeddings

A categorical feature is a feature that does not express a continuous quantity, but rather takes on one of a set of fixed values. Most deep learning models express these feature by turning them into high-dimensional embedding vectors which will be adjusted during model training.

Here we represent each customers and each book by an embedding vector. Initially, these embeddings will take on random values, but during training, we will adjust them so that embeddings of customers and the books they watch end up closer together.

Taking raw categorical features and turning them into embeddings is normally a two-step process:


1.   Build a mapping (called a `"vocabulary"`) that maps each raw values.
2.   Turn these integers into embedding vectors.



In [15]:
# Make a Keras StringLookup layer as the mapping (lookup)
book_id_lookup_layer = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)

# StringLookup layer is a non-trainable layer and its state (the vocabulary)
# must be constructed and set before training in a step called "adaptation".
book_id_lookup_layer.adapt(
    ratings_trainset.map(
        lambda x: x['product_id']
    )
)

print(
    f"Vocabulary[:10] -> {book_id_lookup_layer.get_vocabulary()[:10]}"
    # Vocabulary: ['[UNK]', '405', '655', '13', ...]
    # The vocabulary includes one (or more!) unknown (or "out of vocabulary", OOV)
    # tokens. So the layer can handle categorical values that are not in the
    # vocabulary and the model can continue to learn about and make
    # recommendations even using features that have not been seen during
    # vocabulary construction.
)

print(
    "Mapped integer for book ids: ['-2', '13', '655', 'xxx']\n",
    book_id_lookup_layer(
        ['-2', '13', '655', 'xxx']
    )
)

book_id_embedding_dim = 32
# The larger it is, the higher the capacity of the model, but the slower it is
# to fit and serve and more prone to overfitting.

book_id_embedding_layer = tf.keras.layers.Embedding(
    # Size of the vocabulary
    input_dim=book_id_lookup_layer.vocabulary_size(),
    # Dimension of the dense embedding
    output_dim=book_id_embedding_dim
)
 
# A model that takes raw string feature values (product_id) in and yields embeddings
book_id_model = tf.keras.Sequential(
    [
        book_id_lookup_layer,
        book_id_embedding_layer
    ]
)
 
print("Embeddings for bookd ids: ['-2', '13', '655', 'xxx']\n",book_id_model(['-2', '13', '655', 'xxx']))


customer_id_lookup_layer =  tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
customer_id_lookup_layer.adapt(
    ratings_trainset.map(
        lambda x: x['customer_id']
    )
)

# Same as user_id_embedding_dim to be able to measure the similarity
customer_id_embedding_dim = 32

customer_id_embedding_layer = tf.keras.layers.Embedding(
    input_dim=customer_id_lookup_layer.vocabulary_size(),
    output_dim=customer_id_embedding_dim
)
 
customer_id_model = tf.keras.Sequential(
    [
        customer_id_lookup_layer,
        customer_id_embedding_layer
    ]
)

print(f"Embedding for the customer 52997076:\n {customer_id_model('52997076')}")

Vocabulary[:10] -> ['[UNK]', '043935806X', '0439139597', '0525947647', '0895260174', '0385504209', '0590353403', '0439784549', '0316666343', '1400050308']
Mapped integer for book ids: ['-2', '13', '655', 'xxx']
 tf.Tensor([0 0 0 0], shape=(4,), dtype=int64)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Embeddings for bookd ids: ['-2', '13', '655', 'xxx']
 tf.Tensor(
[[-1.9437671e-03 -3.8034208e-03 -1.0673761e-02 -8.8535249e-05
  -2.5959326e-02  9.6588843e-03  1.9897494e-02  2.9990140e-02
   7.4738637e-03 -4.7060825e-02  1.8053088e-02  3.6847744e-02
  -1.5759788e-02  3.4050200e-02 -1.2041248e-02 -2.0433677e-02
  -2.9981030e-02 -3.7906587e-02  3.9568316e-02 -2.2324467e-02
  -4.4263553e-02 -3.3800088e-02 -3.2531559e-02 -4.6921935e-02
   1.5477095e-02  2.5458846e-02 -7.1270950e-03 -2.9826880e-02
   1.9904528e-02 -2.9718652e-03  7.6395869e-03 -1.8093131e-02]
 [-1.9437671e-03 -3.8034208e-03 -1.0673761e-02 -8.8535249e-05
  -2.5959326e-02  9.6588843e-03  1.9897494e-02  2.9990140e-02
   7.4738637e-03 -4.7060825e-02  1.8053088e-02  3.6847744e-02
  -1.5759788e-02  3.4050200e-02 -1.2041248e-02 -2.0433677e-02
  -2.9981030e-02 -3.7906587e-02  3.9568316e-02 -2.2324467e-02
  -4.4263553e-02 -3.3800088e-02 -3.2531559e-02 -4.6921935e-02
   1.5477095e-02  2.5458846e-02 -7.1270950e-03 -2.9826880e-02
   1

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Embedding for the customer 52997076:
 [ 0.00067756  0.03443528 -0.04296687 -0.03931494  0.02974707 -0.00172361
  0.01440866 -0.0425303  -0.00610284 -0.00090276  0.01798202  0.03616593
 -0.03439077  0.02476938  0.04241036 -0.00179129 -0.01331911 -0.04734996
  0.01387544  0.03550395  0.02973064 -0.02506577 -0.01204579 -0.03935288
 -0.00042962  0.00256138  0.01926417  0.00531001 -0.01102423  0.01505114
  0.02455181 -0.03485101]


### Tokenize textual features and translate them into embeddings
Candidates textual description and users' reviews can be useful especially in a `cold-start` or `long-tail` scenario.

While the Book dataset does not give us rich textual features, we can still use book titles. This may help us capture the fact that books with very similar titles are likely to belong to the same categories.

In [ ]:
# Keras TextVectorization layer transforms the raw texts into `word pieces` and
# map these pieces into tokens.
book_title_vectorization_layer = tf.keras.layers.experimental.preprocessing.TextVectorization()
book_title_vectorization_layer.adapt(
    ratings_trainset.map(
        lambda rating: rating['product_title']
    )
)

# Verify that the tokenization is done correctly
print(
    "Vocabulary[40:50] -> ",
    book_title_vectorization_layer.get_vocabulary()[40:50]
)

print(
    "Vectorized title for 'Shutterbabe:  Adventures in Love and War'\n",
    book_title_vectorization_layer('Shutterbabe:  Adventures in Love and War')
)

book_title_embedding_dim = 32
book_title_embedding_layer = tf.keras.layers.Embedding(
    input_dim=len(book_title_vectorization_layer.get_vocabulary()),
    output_dim=book_title_embedding_dim,
    # Whether or not the input value 0 is a MASK token.
    # Keras TextVectorization layer builds the vocabulary with MASK token.
    mask_zero=True
)

book_title_model = tf.keras.Sequential(
    [
       book_title_vectorization_layer,
       book_title_embedding_layer,
       # each title contains multiple words, so we will get multiple embeddings
       # for each title that should be compressed into a single embedding for
       # the text. Models like RNNs, Transformers or Attentions are useful here.
       # However, averaging all the words' embeddings together is also a good
       # starting point.
       tf.keras.layers.GlobalAveragePooling1D()
    ]
)

## Query and Candidate representation
We are building a [two-tower retrieval model](https://research.google/pubs/pub48840/), a model including two seperate models (towers) one for transforming query raw features to query representation (query tower) and one another for transforming candidate raw features to the same dimensionality candidate representation.

The output tensors of the two models will multiply together (inner product) to give a query-candidate `affinity score` (similarity measure). Higher scores express a better match between the candidate and the query.

In [17]:
# we want to have complex model for candidate model
class CandidateModel(tfrs.models.Model):

    def __init__(self, book_id_model, book_title_model):
      super().__init__()
      self.book_id_model: tf.keras.Model = book_id_model
      self.book_title_model: tf.keras.Model = book_title_model

    def get_model(self):
        return tf.concat([
            self.book_id_model,
            self.book_title_model],axis=1)

# Query tower
query_model = customer_id_model
 
# Candidate tower

# c = CandidateModel(book_id_model,book_title_model)
# candidate_model = c.get_model() 
candidate_model = book_id_model 

print(candidate_model)
print(query_model)

## Build the Retrieval (Candidate Generation) task

It is about selecting an initial set of hundreds of candidates from all possible candidates. The main objective of this model is to efficiently weed out all candidates that the user is not interested in. Because the retrieval model may be dealing with millions of candidates, **it has to be computationally efficient**.

A retrieval system is a model that predicts a set of mbooksovies from the catalogue that the user is likely to watch. So the train set should be expressesing which book the users watched, and which they did not. for example:
```
[
  (('user1', 'With I When 20 Know'), POSITIVE),
  (('user1', 'Mar Man, Women Venus'), NEGATIVE),
  ...
]
```


In [ ]:
# We don't need rating field for the retrieval task
retrieval_ratings_trainset = ratings_trainset.map(
    lambda rating: {
        'customer_id': rating['customer_id'],
        'product_id': rating['product_id'],
    }
)
 
retrieval_ratings_testset = ratings_testset.map(
    lambda rating: {
        'customer_id': rating['customer_id'],
        'product_id': rating['product_id'],
    }
)

In [ ]:
# We are using just `book (product)_id` feature for making the candidates representation
candidates_corpus_dataset = ratings_dataset.map(
    lambda book: book['product_id']
)

### Fit the model using standard Keras routine

In [20]:
factorized_top_k_metrics = tfrs.metrics.FactorizedTopK(
    # dataset of candidate embeddings from which candidates should be retrieved
    candidates=candidates_corpus_dataset.batch(1200).map(
        candidate_model
    )
)

retrieval_task_layer = tfrs.tasks.Retrieval(
    metrics=factorized_top_k_metrics
)

# The task computes the metrics and return the in-batch softmax loss.
# Because the metrics range over the entire candidate set, they are usually much
# slower to compute. Consider setting `compute_metrics=False` in Retrieval
# costructor during training to save the time in computing the metrics.

### Create the training loop
To create an appropriate training loop and train the models we can extend the class `tf.keras.Model` and override the `train_step` and `test_step` functions. [See how](https://keras.io/guides/customizing_what_happens_in_fit/).

However, to keep the focus on modelling and abstract away some of the boilerplate, TFRS exposes `tfrs.models.Model` base class which allows us to compute both training and test losses using the same method. All we need to do is to set up the components in the `__init__` method, and implement the `compute_loss` method, taking in the raw features and returning a loss value. The base model will then take care of creating the appropriate training loop to fit the model.

In [21]:
class RetrievalModel(tfrs.models.Model):
  """Amazon_us_review_Books candidate generation model"""
 
  def __init__(self, query_model, candidate_model, retrieval_task_layer):
    super().__init__()
    self.query_model: tf.keras.Model = query_model
    self.candidate_model: tf.keras.Model = candidate_model
    self.retrieval_task_layer: tf.keras.layers.Layer = retrieval_task_layer
 
 #def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
  def compute_loss(self, features, training=False) -> tf.Tensor:
    query_embeddings = self.query_model(features['customer_id'])
    positive_candidate_embeddings = self.candidate_model(features["product_id"])

    loss = self.retrieval_task_layer(
        query_embeddings,
        positive_candidate_embeddings
        # ,compute_metrics=not training  # To speed up training
    )
    return loss

In [22]:
amazon_book_retrieval_model = RetrievalModel(
    query_model,
    candidate_model,
    retrieval_task_layer
)

optimizer_step_size = 0.1
amazon_book_retrieval_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(
        learning_rate=optimizer_step_size
    )
)

In [ ]:
# Shuffle the training data for each epoch.
# Batch and cache both the training and evaluation data.
# `cache()` method caches the elements in the dataset in memory. To caches data
# in a file pass the `filename` argument to the method: cache(filename='')
# The first time the dataset is iterated over, its elements will be cached
# either in the specified file or in memory. Subsequent iterations will use the
# cached data.
retrieval_cached_ratings_trainset =  retrieval_ratings_trainset.shuffle(3_105_520).batch(128192).cache()
retrieval_cached_ratings_testset =  retrieval_ratings_testset.batch(1281926).cache()
 
num_epochs = 10 
history = amazon_book_retrieval_model.fit(
    retrieval_cached_ratings_trainset,
    validation_data=retrieval_cached_ratings_testset,
    validation_freq=1,
    epochs=num_epochs
)

In [ ]:
scann_layer = tfrs.layers.factorized_top_k.ScaNN(
    amazon_book_retrieval_model.query_model
)

scann_layer.index(
    candidates_corpus_dataset.batch(100).map(
        amazon_book_retrieval_model.candidate_model
    ),
    candidates_corpus_dataset
)

user_id = '42'
afinity_scores, book_ids = scann_layer(
    tf.constant([user_id])
)

print(f"Recommendations for user {user_id} using ScaNN: {book_ids[0, :5]}")

In [ ]:
class RankingModel(tfrs.models.Model):
  """Book dataset ranking model"""

  def __init__(self, query_model, candidate_model):
    super().__init__()

    self.query_model: tf.keras.Model = query_model
    self.candidate_model: tf.keras.Model = candidate_model
    self.rating_model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ]
    )
    self.ranking_task_layer: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )


  def compute_loss(self, features, training=False) -> tf.Tensor:
    query_embeddings = self.query_model(features['customer_id'])
    candidate_embeddings = self.candidate_model(features["product_id"])
    rating_predictions = self.rating_model(
        tf.concat(
            [query_embeddings, candidate_embeddings],
            axis=1
        )
        # We could use `tf.keras.layers.Concatenate(axis=1)([x, y])`
    )

    loss = self.ranking_task_layer(
        predictions=rating_predictions,
        labels=features["user_rating"]
    )
    return loss

In [ ]:
books_ranking_model = RankingModel(query_model, candidate_model)

optimizer_step_size = 0.1
books_ranking_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(
        learning_rate=optimizer_step_size
    )
)

In [ ]:
ranking_ratings_trainset = ratings_trainset.shuffle(100_000).batch(8192).cache()
ranking_ratings_testset = ratings_testset.batch(4096).cache()

history = books_ranking_model.fit(
    ranking_ratings_trainset,
    validation_data=ranking_ratings_testset,
    validation_freq=1,
    epochs=5
)